<a href="https://colab.research.google.com/github/tamaskecskemeti/financial_nlp/blob/main/nlp_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# install required packages written in requirements
!pip install -r requirements.txt

In [8]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, set_seed, GPTNeoXForCausalLM, AutoTokenizer
import torch
import torch.nn.functional as F
import itertools

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [9]:
set_seed(42)

In [10]:
# create the pretrained tokenizer and the model from HuggingFace
tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/mGPT")
model = GPT2LMHeadModel.from_pretrained("sberbank-ai/mGPT")

In [11]:
# some text to test the model
text = "The Turing test, originally called the imitation game by Alan Turing in 1950 is a test of a machine's ability to exhibit intelligent behaviour equivalent to, or indistinguishable from, that of a human."

# encode the text
input_ids = tokenizer.encode(text, return_tensors="pt")

out = model.generate(input_ids,
                     max_new_tokens=50,
                     num_beams=5,
                     no_repeat_ngram_size=4,
                     top_k=50,
                     do_sample=True,
                     top_p=0.9,
                     temperature=1,
                     early_stopping=True,
                     pad_token_id=tokenizer.eos_token_id)

# decode the generated text and write it out
generated_text = list(map(tokenizer.decode, out))[0]
print(generated_text)

The Turing test, originally called the imitation game by Alan Turing in 1950 is a test of a machine's ability to exhibit intelligent behaviour equivalent to, or indistinguishable from, that of a human. The Turing test was first published in 1951, and was used to test the Turing machine.
The Turing test was originally developed by Alan Turing and published in 1951. It was originally intended to test Turing's ability to imitate human behavior. The test


In [12]:
# import another model to compare the generated texts
tokenizer_2 = GPT2Tokenizer.from_pretrained("gpt2")
model_2 = GPT2LMHeadModel.from_pretrained("gpt2")

In [13]:
# encode the text
input_ids_2 = tokenizer_2.encode(text, return_tensors="pt")

# run the model on the encoded text
out_2 = model_2.generate(input_ids_2,
                         max_new_tokens=50,
                         num_beams=5,
                         no_repeat_ngram_size=4,
                         top_k=50,
                         do_sample=True,
                         top_p=0.9,
                         temperature=1,
                         early_stopping=True,
                         pad_token_id=tokenizer.eos_token_id)

# decode the generated text and write it out
generated_text_2 = list(map(tokenizer_2.decode, out_2))[0]
print(generated_text_2)

The Turing test, originally called the imitation game by Alan Turing in 1950 is a test of a machine's ability to exhibit intelligent behaviour equivalent to, or indistinguishable from, that of a human.

In the test, the Turing test is used to determine whether a computer is capable of performing a Turing test.

The Turing test was first used in the 1970s to test the ability of a computer to perform a Turing test, but


In [14]:
# rouge scores for a reference/generated sentence pair
# source google seq2seq source code.

# supporting function
def _split_into_words(sentences):
  """Splits multiple sentences into words and flattens the result"""
  return list(itertools.chain(*[_.split(" ") for _ in sentences]))

# supporting function
def _get_word_ngrams(n, sentences):
  """Calculates word n-grams for multiple sentences.
  """
  assert len(sentences) > 0
  assert n > 0

  words = _split_into_words(sentences)
  return _get_ngrams(n, words)

# supporting function
def _get_ngrams(n, text):
  """Calcualtes n-grams.
  Args:
    n: which n-grams to calculate
    text: An array of tokens
  Returns:
    A set of n-grams
  """
  ngram_set = set()
  text_length = len(text)
  max_index_ngram_start = text_length - n
  for i in range(max_index_ngram_start + 1):
    ngram_set.add(tuple(text[i:i + n]))
  return ngram_set

def rouge_n(reference_sentences, evaluated_sentences, n=2):
  """
  Args:
    evaluated_sentences: The sentences that have been picked by the summarizer
    reference_sentences: The sentences from the referene set
    n: Size of ngram.  Defaults to 2.
  Returns:
    recall rouge score(float)
  Raises:
    ValueError: raises exception if a param has len <= 0
  """
  if len(evaluated_sentences) <= 0 or len(reference_sentences) <= 0:
    raise ValueError("Collections must contain at least 1 sentence.")

  evaluated_ngrams = _get_word_ngrams(n, evaluated_sentences)
  reference_ngrams = _get_word_ngrams(n, reference_sentences)
  reference_count = len(reference_ngrams)
  evaluated_count = len(evaluated_ngrams)

  # gets the overlapping ngrams between evaluated and reference
  overlapping_ngrams = evaluated_ngrams.intersection(reference_ngrams)
  overlapping_count = len(overlapping_ngrams)

  # handle edge case. This isn't mathematically correct, but it's good enough
  if evaluated_count == 0:
    precision = 0.0
  else:
    precision = overlapping_count / evaluated_count

  if reference_count == 0:
    recall = 0.0
  else:
    recall = overlapping_count / reference_count

  f1_score = 2.0 * ((precision * recall) / (precision + recall + 1e-8))

  # just returning recall count in rouge, useful for our purpose
  return recall

In [15]:
# the reference text is used to evaluate the generated text
ref_text = "The Turing test, originally called the imitation game by Alan Turing in 1950 is a test of a machine's ability to exhibit intelligent behaviour equivalent to, or indistinguishable from, that of a human. Turing proposed that a human evaluator would judge natural language conversations between a human and a machine designed to generate human-like responses. The evaluator would be aware that one of the two partners in conversation was a machine, and all participants would be separated from one another. The conversation would be limited to a text-only channel, such as a computer keyboard and screen, so the result would not depend on the machine's ability to render words as speech. If the evaluator could not reliably tell the machine from the human, the machine would be said to have passed the test. The test results would not depend on the machine's ability to give correct answers to questions, only on how closely its answers resembled those a human would give. Since the Turing test is a test of indistinguishability in performance capacity, the verbal version generalizes naturally to all of human performance capacity, verbal as well as nonverbal (robotic)."

In [16]:
# the rouge value can be between 0 and 1. The higher value is better
print(rouge_n(ref_text, generated_text))

0.5365853658536586


In [17]:
# the rouge value can be between 0 and 1. The higher value is better
print(rouge_n(ref_text, generated_text_2))

0.5528455284552846


# Hungarian model

In [18]:
# some text in hungarian to test the model
text_hu = "A Turing-teszt, amelyet eredetileg Alan Turing 1950-ben imitációs játéknak nevezett el, egy olyan teszt, amely azt vizsgálja, hogy egy gép képes-e az emberi viselkedéssel egyenértékű vagy attól megkülönböztethetetlen intelligens viselkedést tanúsítani."

# encode the text
input_ids_hu = tokenizer.encode(text_hu, return_tensors="pt")

# run the model on the encoded text
out_hu = model.generate(input_ids_hu,
                     max_new_tokens=40,
                     num_beams=5,
                     no_repeat_ngram_size=4,
                     top_k=50,
                     do_sample=True,
                     top_p=0.9,
                     early_stopping=True,
                     pad_token_id=tokenizer.eos_token_id)

In [19]:
# decode the generated hungarian text and write it out
generated_text_hu = list(map(tokenizer.decode, out_hu))[0]
print(generated_text_hu)

A Turing-teszt, amelyet eredetileg Alan Turing 1950-ben imitációs játéknak nevezett el, egy olyan teszt, amely azt vizsgálja, hogy egy gép képes-e az emberi viselkedéssel egyenértékű vagy attól megkülönböztethetetlen intelligens viselkedést tanúsítani.
A tesztet az 1960-as évek elején kezdték kidolgozni, és azóta számos kísérletet hajtottak végre. A teszt az emberi agy működését


In [20]:
# import another model to compare the generated texts
model_2_hu = GPT2LMHeadModel.from_pretrained("NYTK/PULI-GPT-2")
tokenizer_2_hu = AutoTokenizer.from_pretrained("NYTK/PULI-GPT-2")

(…)NYTK/PULI-GPT-2/resolve/main/config.json:   0%|          | 0.00/755 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.47G [00:00<?, ?B/s]

(…)GPT-2/resolve/main/tokenizer_config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

(…)/NYTK/PULI-GPT-2/resolve/main/vocab.json:   0%|          | 0.00/942k [00:00<?, ?B/s]

(…)/NYTK/PULI-GPT-2/resolve/main/merges.txt:   0%|          | 0.00/601k [00:00<?, ?B/s]

(…)K/PULI-GPT-2/resolve/main/tokenizer.json:   0%|          | 0.00/2.39M [00:00<?, ?B/s]

(…)T-2/resolve/main/special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

In [21]:
# encode the text
input_ids_2_hu = tokenizer_2_hu(text_hu, return_tensors="pt").input_ids

# run the model on the encoded text
out_2_hu = model_2_hu.generate(input_ids_2_hu,
                     max_new_tokens=40,
                     num_beams=5,
                     no_repeat_ngram_size=4,
                     top_k=50,
                     do_sample=True,
                     top_p=0.9,
                     early_stopping=True,
                     pad_token_id=tokenizer.eos_token_id)

In [24]:
# decode the generated hungarian text and write it out
generated_text_2_hu = list(map(tokenizer.decode, out_hu))[0]
print(generated_text_2_hu)

A Turing-teszt, amelyet eredetileg Alan Turing 1950-ben imitációs játéknak nevezett el, egy olyan teszt, amely azt vizsgálja, hogy egy gép képes-e az emberi viselkedéssel egyenértékű vagy attól megkülönböztethetetlen intelligens viselkedést tanúsítani. A tesztet a számítógépes programozás egyik alapvető eleme, a Turing-algoritmus segítségével végezték el.
Története.
A teszt


In [22]:
# the reference text is used to evaluate the generated text
ref_text_hu = "A Turing-teszt, amelyet eredetileg Alan Turing 1950-ben imitációs játéknak nevezett el, egy olyan teszt, amely azt vizsgálja, hogy egy gép képes-e az emberi viselkedéssel egyenértékű vagy attól megkülönböztethetetlen intelligens viselkedést tanúsítani. Turing azt javasolta, hogy egy emberi értékelő értékelje az ember és egy emberhez hasonló válaszok generálására tervezett gép közötti természetes nyelvű beszélgetéseket. Az értékelő tisztában lenne azzal, hogy a két beszélgetőpartner közül az egyik egy gép, és a résztvevők el lennének választva egymástól. A beszélgetés kizárólag szöveges csatornára korlátozódna, például számítógépes billentyűzetre és képernyőre, így az eredmény nem függne attól, hogy a gép képes-e a szavakat beszédként megjeleníteni. Ha az értékelő nem tudná megbízhatóan megkülönböztetni a gépet az embertől, akkor a gép átmenne a teszten. A teszt eredménye nem függne attól, hogy a gép képes-e helyes válaszokat adni a kérdésekre, csak attól, hogy a válaszai mennyire hasonlítanak az emberi válaszokhoz. Mivel a Turing-teszt a teljesítőképesség megkülönböztethetetlenségének tesztje, a verbális változat természetesen általánosítható az emberi teljesítőképesség egészére, a verbális és a nem verbális (robotikus) teljesítőképességre egyaránt."

In [25]:
# the rouge value can be between 0 and 1. The higher value is better
print(rouge_n(ref_text_hu, generated_text_hu))

0.5573248407643312


In [26]:
# the rouge value can be between 0 and 1. The higher value is better
print(rouge_n(ref_text_hu, generated_text_2_hu))

0.5700636942675159
